In [4]:
!export PYTHONPATH="$PYTHONPATH:../workspace_files/"
from search import *

# A. Representación del problema

### Estado del problema
Para la representación de estados, usamos una lista de listas de stacks de booleanos (representacion de pila - representacion de lista de pilas) (`List[Stack[bool]]`). La razón es que sólo necesitamos saber el número de pilas del muelle 1, para deducir cuáles son las pilas destino.

Un `bool` representa a un contenedor, donde `True` son los contenedores objetivo y `False` el resto de contenedores. Un `Stack[bool]` representa una pila de contenedores, las cuales recogemos en una lista para representar un muelle como `List[Stack[bool]]`. Como tenemos más de un muelle, los guardamos en una lista también, resultando en nuestra definición `Estado`.

Como queremos poder guardar nuestros estados en diccionarios y conjuntos, los hacemos hasheables implementando el metamétodo `__hash__` (Nota: esto es para después poder aplicar los métodos del `search.py`, que utilizan `Set`)

In [5]:
from typing import *

Stack = List
def hash_list(ls: list):
    return hash(tuple(hash_list(x) if type(x) == list else x for x in ls))

class Estado(List[Stack[bool]]):
    def __hash__(self): 
        return hash_list(self)
# Lista de muelles, cada muelle tiene una lista de stacks, con booleanos que indican si el contenedor es objetivo o no

### Acciones del problema
Las acciones que podemos tomar se basan en reorganizar los contenedores entre pilas. El uso de las cintas solo nos permite intercambiar contenedores entre muelles, que una grúa por si sola no pueda hacerlo.

Por lo tanto, nos basta con identificar la pila de la que queremos sacar el contenedor, que llamaremos `origen`, y la pila en la que queremos colocarlo, que llamaremos `destino`. Para identificar una pila, usamos su índice dentro de la lista.

In [6]:
from collections import namedtuple

# Tipo                   Nombre        Atributos
Movimiento = namedtuple("Movimiento",["origen","destino"])

### El estado objetivo

Nuestro estado objetivo consiste en llevar todos los contenedores objetivo al primer muelle, donde deben estar colocados de tal manera que no tengan por encima ningún contenedor que no sea objetivo.

<!--
Comentario de representación antigua del problema
### Notas de interés
 - En realidad el único muelle que es necesario distinguir es el primero, pues es el que distingue entre estados objetivos y no objetivo. Si hubieran más de dos muelles, podemos considerar que el resto de muelles que no son el primero son en realidad uno. Sin embargo, el realizar esta simplificación no reduce la información que necesitamos almacenar del problema, por lo que he decidido mantener la representación así para dotarle de mayor estructura. 
  Esto en un caso real nos daría mayor flexibilidad a la hora de adaptar el problema a condiciones diferentes. ¿Y si no hubiera cintas entre todos los muelles? Entonces tendríamos que definir una lista de cintas en el inicio del problema, y asegurarnos de no mover contenedores entre muelles donde no haya una cinta. Realizar ese cambio sería sencillo con la estructura actual, y no tan sencillo si mezclamos los distintos muelles.-->

In [7]:
from copy import deepcopy
import math


forall = lambda condition, ls: all(map(condition,ls))

class ProblemaContenedor(Problem):
    def __init__(self, initial: Estado, num_pilas_muelle_1: int, altura_maxima: int = 10):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        super().__init__(initial)
        self.altura_maxima = altura_maxima
        self.esta_llena = lambda pila: len(pila) == self.altura_maxima

        ## Lo que nos importa realmente es el número de pilas del muelle 1
        self.num_pilas_muelle_1 = num_pilas_muelle_1

        # Contamos cuantos contenerdores objetivos hay en total, que son los que tenemos que poner en el muelle 1
        self.num_objetivo = 0
        
        for pila in initial:
            self.num_objetivo += sum(pila)

    def actions(self, state: Estado) -> List[Movimiento]:
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        actions = []
        
        pilas_no_vacias = { i for i, pila in enumerate(state) if len(pila) > 0}
        pilas_no_llenas = { i for i, pila in enumerate(state) if len(pila) < self.altura_maxima}

        # Para cada pila no vacia, moverlo a otra pila no llena que no sea ella misma
        for origen in pilas_no_vacias:
            for destino in pilas_no_llenas:
                if origen == destino: continue
                actions.append(Movimiento(origen=origen,destino=destino))
                
        return actions

    def result(self, state: Estado, action: Movimiento):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        origen = action.origen
        destino = action.destino

        res = deepcopy(state)
        el = res[origen].pop()
        res[destino].append(el)

        return res

    def goal_test(self, state: Estado):
        """Return True if the state is a goal."""

        # Todos los contenedores objetivo estan en el primer muelle de descarga
        obj_primer_muelle = 0
        for pila in state[0:self.num_pilas_muelle_1]:
            obj_primer_muelle += ProblemaContenedor.num_objetivos_encima(pila)
        
        # Es objetivo cuando todos los contenedores objetivos estan en el primer muelle de descarga, por encima
        return self.num_objetivo == obj_primer_muelle
    
    @staticmethod
    def num_objetivos_encima(pila: Stack):
        """Sacar el numero de contenedores objetivos que hay encima de una pila, suponiendo que el cima es el final"""
        i = len(pila) - 1
        result = 0
        while i >= 0 and pila[i]:
            i -= 1
            result += 1
        
        return result
    
    def show_estado(self, state: Estado):
        def show_pila(pila: Stack):
            return "|".join(map(lambda x : "░░" if x else "██",pila))
        num_width = math.ceil(math.log(len(state),10))
        print("\n".join(map(lambda x : str(x[0]).rjust(num_width," ")+": " +show_pila(x[1]), enumerate(state[0:self.num_pilas_muelle_1]))))
        print()
        print("\n".join(map(lambda x : str(x[0]+self.num_pilas_muelle_1).rjust(num_width," ")+": " +show_pila(x[1]), enumerate(state[self.num_pilas_muelle_1:]))))

    
    def heuristica(self, node):
        state = node.state
        coste = 0

        # Todos los contenedores objetivo que no están en el primer muelle de descarga
        obj_primer_muelle = 0
        for pila in state[0:self.num_pilas_muelle_1]:
            obj_primer_muelle += ProblemaContenedor.num_objetivos_encima(pila)

        coste = self.num_objetivo - obj_primer_muelle

        # Todos los contenedores no objetivo entre el contenedor objetivo más bajo y la cima
        for i, pila in enumerate(state):
            es_muelle1 = i < self.num_pilas_muelle_1
            num_objetivo_encima = ProblemaContenedor.num_objetivos_encima(pila)
            num_objetivo_total = sum(pila)
            if num_objetivo_total > 0 and num_objetivo_total != num_objetivo_encima:
                ## Entonces tiene contenedores objetivos que esta por debajo, que hay que sacarlo
                contenedor_objetivo_mas_bajo = pila.index(True)

                ## Si no estamos en muelle 1, entonces tiene mayor coste sacarlo, lo 
                coste += (1 if es_muelle1 else 3) * len(pila[contenedor_objetivo_mas_bajo:]) - sum(pila[contenedor_objetivo_mas_bajo:])

        return coste

Con esta representación, nuestro estado inicial (el del ejemplo del PDF) sería la siguiente:

A continuación hemos realizado algunas pruebas con los operadores definidos por nosotros.

In [8]:
initial = Estado([
     # Muelle 1
        [False, True,],
        [False, False,],
        [False,],
     # Muelle 2
        [True, False,],
        [False,True, False,],
        [False,],
])

In [9]:
prob = ProblemaContenedor(initial, 3)
prob.show_estado(initial)

0: ██|░░
1: ██|██
2: ██

3: ░░|██
4: ██|░░|██
5: ██


In [10]:
acts = prob.actions(initial)
acts

[Movimiento(origen=0, destino=1),
 Movimiento(origen=0, destino=2),
 Movimiento(origen=0, destino=3),
 Movimiento(origen=0, destino=4),
 Movimiento(origen=0, destino=5),
 Movimiento(origen=1, destino=0),
 Movimiento(origen=1, destino=2),
 Movimiento(origen=1, destino=3),
 Movimiento(origen=1, destino=4),
 Movimiento(origen=1, destino=5),
 Movimiento(origen=2, destino=0),
 Movimiento(origen=2, destino=1),
 Movimiento(origen=2, destino=3),
 Movimiento(origen=2, destino=4),
 Movimiento(origen=2, destino=5),
 Movimiento(origen=3, destino=0),
 Movimiento(origen=3, destino=1),
 Movimiento(origen=3, destino=2),
 Movimiento(origen=3, destino=4),
 Movimiento(origen=3, destino=5),
 Movimiento(origen=4, destino=0),
 Movimiento(origen=4, destino=1),
 Movimiento(origen=4, destino=2),
 Movimiento(origen=4, destino=3),
 Movimiento(origen=4, destino=5),
 Movimiento(origen=5, destino=0),
 Movimiento(origen=5, destino=1),
 Movimiento(origen=5, destino=2),
 Movimiento(origen=5, destino=3),
 Movimiento(or

In [11]:
prob.show_estado(initial)
print('\n',acts[1],'\n')
prob.show_estado(prob.result(initial, acts[1]))

0: ██|░░
1: ██|██
2: ██

3: ░░|██
4: ██|░░|██
5: ██

 Movimiento(origen=0, destino=2) 

0: ██
1: ██|██
2: ██|░░

3: ░░|██
4: ██|░░|██
5: ██


In [12]:
final = prob.result(initial, acts[1])
prob.goal_test(final)

False

# B. Resolución del problema
Podemos observar que éste es un clásico ejemplo del problema de búsqueda. En particular, vemos que cada estado tiene múltiples acciones que se pueden aplicar sobre él, lo cual nos indica que hacer una búsqueda en anchura probablemente sea una mala idea, ya que **el número de estados hijo es muy elevado**. Puesto que cada acción tiene una acción inversa, es buena idea llevar algún tipo de control de repetidos, para evitar volver a estados ya procesados.

## Búsqueda ciega 
Supongamos que $r$ es el factor de ramificación máximo, $p$ es la profundidad del camino a la solución, $m$ la profundidad máxima

 - **Búsqueda en anchura** 
   En cada paso se expande el nodo abierto más antiguo
 
    - Tiempo: $O(r^p)$
    - Espacio: $O(r^p)$

   Es **completa** y **óptima** _si el coste de los operadores es uniforme_
 
 - **Búsqueda de coste uniforme** 
   En cada paso se expande el nodo abierto con menor coste de camino hasta llegar a él
  
    - Tiempo: $O(r^p)$
    - Espacio: $O(r^p)$
  
   Es **completa** y **óptima** _si los operadores tiene coste positivo_ (No se aplica a nuestro caso)
 
 - **Búsqueda en profundidad**
   En cada paso se expande el nodo abierto más reciente

    - Tiempo: $O(r^m)$
    - Espacio: $O(r·m)$

   Supone una mejora considerable con respecto a la búsqueda en profundidad en cuanto a espacio. En tiempo puede ser peor, pero suele dar mejores resultados cuando hay varios posibles caminos a un estado objetivo
 
 - **Búsqueda en profundidad limitada**
   Una búsqueda en profundidad con un límite $L$ de profundidad (Evita descender indefinidamente)

    - Tiempo: $O(r^L)$
    - Espacio: $O(r·L)$
 
 - **Búsqueda en profundidad iterativa**  
   Una búsqueda en profundidad limitada, donde variamos el límite $L$ con una función $f:\mathbb{N}\to\mathbb{R}$ en distintas iteraciones

    - Tiempo: $O(f^{-1}(m)·r^m)$
    - Espacio: $O(r·m·f^{-1}(m))$
 
   Evita el problema de la elección del límite.  
   Si $f(n) = n$ incluimos un factor lineal en la complejidad, pero si tomamos $f(n) = 2^n$, el factor es logarítmico
 
 - **Búsqueda bidireccional**
   Se realizan dos búsquedas simultáneas, una desde el estado inicial y otra desde el estado final. Se termina la búsqueda cuando algun estado se haya alcanzado desde ambas búsquedas

    - Tiempo: $O(t_1+t_2)$ con $t_i$ la complejidad en tiempo de la búsqueda $i$-ésima
    - Espacio: $O(e_1+e_2)$ con $e_i$ la complejidad en espacio de la búsqueda $i$-ésima

   La motivación viene del hecho que $r^{\frac{p}{2}} + r^{\frac{p}{2}} \le r^p$. Normalmente una de las búsquedas es una búsqueda completa (coste uniforme o en anchura), y otra una búsqueda que no requierea mucho control de repetidos. Uno de sus inconvenientes es que necesitamos definir explícitamente un estado final, y no una propiedad que cumplen los estados finales.
   

Debido a que este es un problema con una ramificación bastante elevada, pensamos que sería mejor usar una búsqueda en profundidad o en anchura con control de repetidos. 

Podríamos hacer una búsqueda bidireccional, pero solo si proporcionamos además un estado final. Tendríamos que tener en cuenta también que el estado final no es único, por lo que podríamos terminar la búsqueda si encontramos un estado final distinto al especificado. Sin embargo, esta forma de hacer búsuqeda bidireccional no está implementeada en AIMA, así que no la consideramos.

Observando los resultados de ejecución, podemos ver que el primer nivel de nuestro estado inicial ya posee 30 hijos, lo cual sugiere que la búsqueda ciega no sería lo óptimo, y lo ideal sería que tuviesemos alguna heurística buena para explorar solo una cantidad diminuta de ellos.

Allí es donde entramos en la **búsqueda heurística**. La idea es explorar los nodos más prometedores de acuerdo con el valor que devuelve la función heurística. Usa una cola de prioridad ordenada por el valor heurístico de cada nodo, y explora el mejor nodo

De las que hemos visto en clase:

- **Búsqueda voraz**: la función heurística considera exclusivamente lo que falta para llegar a la solución. Es decir, devuelve el coste mínimo estimado para llegar a una solución **a partir de un nodo n** 
- **Búsqueda A***: La heurística considera el **coste total** estimado del camino desde el nodo inicial a un nodo objetivo pasando por el nodo n

# C: Buscando soluciones

Resuelve el problema razonando convenientemente el comportamiento de distintos algoritmos (a elegir)

Basandonos en el análisis anterior, podemos concluir que las búsquedas ciegas no son muy útiles aquí:

In [14]:
%%time
breadth_first_graph_search(ProblemaContenedor(initial, 3))

CPU times: user 30.8 s, sys: 12.2 ms, total: 30.8 s
Wall time: 30.9 s


<Node [[False, True, True, True], [False, False, False, False], [False], [], [False], [False]]>

In [56]:
%%time
depth_limited_search(ProblemaContenedor(initial, 3))

6.4 µs ± 109 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [57]:
%%time
iterative_deepening_search(ProblemaContenedor(initial, 3))

7.7 µs ± 43.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Crear una heurística

Se ve claramente como podemos relajar el problema para calcular el coste de la solución rápidamente dado un estado. 

Si **quitamos las restricciones de que haya un límite en altura en las pilas**, podemos considerar que podemos mover siempre los contenedores que deseamos situar en el primer muelle en los puestos superiores.

Nuestra heurística viene dada por la suma de los contenedores que faltan por colocar en el primer muelle más la suma de los contenedores que se encuentran por encima del contenedor objetivo más abajo en la cada una de las pilas. Si la pila es del muelle 1, entonces multiplicamos el número de contenedores por 1 y si pertenece a otro muelle, multiplicamos por un factor 3. 

En el caso de que todos los contenedores que se encuentran encima del contenedor objetivo más abajo en la pila son a su vez todos objetivos, y esta pila pertenece al muelle 1, entonces no los consideramos; ya que no tenemos que realizar ningún movimiento. Esta heurística es admisible porque para poder llegar a un contedor siempre debemos quitar los que tiene encima, considerando además las condiciones adicionales del problema (como el límite de las pilas). 

In [16]:
def heuristica(self: ProblemaContenedor, state: Estado):
    coste = 0

    # Todos los contenedores objetivo que no están en el primer muelle de descarga
    obj_primer_muelle = 0
    for pila in state[0:self.num_pilas_muelle_1]:
        obj_primer_muelle += ProblemaContenedor.num_objetivos_encima(pila)

    coste = self.num_objetivo - obj_primer_muelle

    # Todos los contenedores no objetivo entre el contenedor objetivo más bajo y la cima
    for i, pila in enumerate(state):
        es_muelle1 = i < self.num_pilas_muelle_1
        num_objetivo_encima = ProblemaContenedor.num_objetivos_encima(pila)
        num_objetivo_total = sum(pila)
        if num_objetivo_total > 0 and num_objetivo_total != num_objetivo_encima:
            ## Entonces tiene contenedores objetivos que esta por debajo, que hay que sacarlo
            contenedor_objetivo_mas_bajo = pila.index(True)

            ## Si no estamos en muelle 1, entonces tiene mayor coste sacarlo, lo 
            coste += (1 if es_muelle1 else 3) * len(pila[contenedor_objetivo_mas_bajo:]) - sum(pila[contenedor_objetivo_mas_bajo:])

    return coste

In [19]:
class Problema_con_Analizados(Problem):
    """Es un problema que se comporta exactamente igual que el que recibe al
       inicializarse, y además incorpora unos atributos nuevos para almacenar el
       número de nodos analizados durante la búsqueda. De esta manera, no
       tenemos que modificar el código del algoritmo de búsqueda."""

    def __init__(self, problem):
        self.initial = problem.initial
        self.problem = problem
        self.analizados = 0
        self.goal = problem.goal

    def actions(self, estado):
        return self.problem.actions(estado)

    def result(self, estado, accion):
        return self.problem.result(estado, accion)

    def goal_test(self, estado):
        self.analizados += 1
        return self.problem.goal_test(estado)

    def coste_de_aplicar_accion(self, estado, accion):
        return self.problem.coste_de_aplicar_accion(estado, accion)

In [20]:
def resuelve_contenedor(p, algoritmo, h=None):
    pa = Problema_con_Analizados(p)
    if h:
        sol = algoritmo(pa, h).solution()
    else:
        sol = algoritmo(pa).solution()
    print(f"Solución: {sol}")
    print(f"Algoritmo: {algoritmo.__name__}")
    if h:
        print(f"Heurística: {h.__name__}")
    print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol), pa.analizados))

In [21]:
resuelve_contenedor(prob, astar_search, prob.heuristica)

Solución: [Movimiento(origen=3, destino=5), Movimiento(origen=4, destino=2), Movimiento(origen=3, destino=2), Movimiento(origen=4, destino=2)]
Algoritmo: astar_search
Heurística: heuristica
Longitud de la solución: 4. Nodos analizados: 7


Sería posible calcular la heurística de un estado junto con el estado en sí mismo, es decir, actualizar la heurística de un estado con una transición para obtener la heurística del estado resultante. Así, el coste de calcular la heurística sería constante, y no $O(n)$, donde $n$ es el número de pilas. En nuestro caso, el coste de calcular la heurística es prácticamente lineal. Sin embargo, hay posibilidad de mejora.

Para ello, deberemos actualizar la heurística cada vez que apliquemos una acción que mueva un contenedor entre dos muelles distintos, donde uno de ellos es el primero.

Si se está moviendo un contenedor del primer muelle, decrementaremos la heurística si no se trata de un contenedor objetivo y la pila de la que se sacó tiene algún contenedor objetivo. Si el contenedor es un contenedor objetivo, incrementaremos la heurística, pues se trata de un contenedor objetivo que estaba en la cima y que hemos sacado del muelle donde debería estar.

Si se está moviendo un contenedor al primer muelle, incrementaremos la heurística si no se trata de un contenedor objetivo y la pila en la que se va a introducir tiene algún contenedor objetivo. Si se trata de un contenedor objetivo, decrementamos la heurística. No importa en que pila lo coloquemos, será el primero, así que no tendrá contenedores no objetivo encima suyo.

In [15]:
# Lista de pilas con booleanos que indican si el contenedor es objetivo o no
Heuristica = int
EstadoConHeuristica = Tuple[Estado, Heuristica]

num_contenedores_objetivo = sum

# Ejemplo:
#  forall(lambda x : x % 2 == 0, range(0,10,2)) == True

class ProblemaContenedorConHeuristica(ProblemaContenedor):
    def __init__(self, initial: Estado, num_pilas_muelle_1: int, altura_maxima: int = 10):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = (initial,heuristica(initial))
        self.altura_maxima = altura_maxima
        self.num_pilas_muelle_1 = num_pilas_muelle_1 
        self.esta_llena = lambda pila: len(pila) == self.altura_maxima

    def actions(self, state: EstadoConHeuristica) -> List[Movimiento]:
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        pilas, _ = state
        return super().actions(pilas)

    def result(self, state: EstadoConHeuristica, action: Movimiento) -> EstadoConHeuristica:
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        # TODO: MODIFICAR ESTADOS
        pilas, heuristica = state
        res = super().result(pilas, action)
        pila_origen = pilas[action.origen] 
        pila_destino = pilas[action.destino]

        contenedor = pila_origen[-1]
        
        # Actualizar heuristica
        if action.origen < self.num_pilas_objetivo != action.destino < self.num_pilas_objetivo:
            if action.origen < self.num_pilas_objetivo:
                if contenedor:
                    heuristica += 1
                elif num_contenedores_objetivo(pila_origen) > 0:
                    heuristica += -1
            elif action.destino < self.num_pilas_objetivo:
                if contenedor:
                    heuristica += -1
                elif num_contenedores_objetivo(pila_destino) > 0:
                    heuristica += 1

        return res, heuristica

    def heuristica(self, state: EstadoConHeuristica) -> Heuristica:
        """Return the heuristic of a given state"""
        return state[1]

    def goal_test(self, state: EstadoConHeuristica) -> bool:
        """Return True if the state is a goal."""
        return self.heuristica(state) == 0

Observamos que en el proceso de actualizar la heurística, en realidad estamos haciendo una valoración del movimiento dado el estado actual, y añadiendolo al coste esperado hasta el momento. Por lo tanto, podemos generalizar esta lógica sobreescribiendo el método `path_cost` de `search.Problem`, para poder usar esta heurística en la búsqueda de coste uniforme

In [16]:
class ProblemaContenedorConHeuristicaYCosteDeCaminos(ProblemaContenedor):
    def action_cost(self, state: EstadoConHeuristica, action: Movimiento):
        """Devuelve el coste de realizar la acción `action` sobre el
        estado `state`"""
        muelles, _heuristica = state
        pila_origen = muelles[action.origen]
        pila_destino = muelles[action.destino]
        contenedor = pila_origen[-1]

        if action.origen < self.num_pilas_objetivo != action.destino < self.num_pilas_objetivo:
            if action.origen < self.num_pilas_objetivo:
                if contenedor:
                    return 1
                elif num_contenedores_objetivo(pila_origen) > 0:
                    return -1
            elif action.destino < self.num_pilas_objetivo:
                if contenedor:
                    return -1
                elif num_contenedores_objetivo(pila_destino) > 0:
                    return 1
        return 0
    
    def path_cost(self, c: Heuristica, state1: EstadoConHeuristica, action: Movimiento, state2: EstadoConHeuristica):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2. If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + self.action_cost(state1, action)
    
    def result(self, state: EstadoConHeuristica, action: Movimiento) -> EstadoConHeuristica:
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        muelles, heuristica = state
        
        heuristica += self.action_cost(state, action)

        res = super().result(muelles, action)
        return res, heuristica

# D. Resolver otras instancias del problema cambiando la situación inicial y final.
 - Incluir nuevos contenedores y/o contenedores objetivo.
 - Modificar la altura de las pilas de los muelles. Se puede especificar alturas
distintas para cada muelle pero siempre la misma altura para todas las pilas
de un muelle.

Vamos a añadir diferentes pilas, pero representamos como si solo hubiera dos muelles, pues así es como hemos decidido representar nuestro problema. Presentamos a continuación cinco ejemplos diferentes. Los tres primeros tratan de representar situaciones reales y los dos último situaciones extremas. Podemos observar que el algoritmo actúa eficientemente en los tres primeros casos y en el último, pero en el caso de una pila con muchos contenedores con uno de ellos True el programa tarda mucho en dar con una solución correcta.

In [20]:
initial2 = Estado([
     # Muelle 1
        [False, True,],
        [False, False,],
        [False,],
     # Muelle 2
        [True, False,],
        [False,False,True,],
        [True,False,],
])

In [34]:
prob2 = ProblemaContenedor(initial2, 3)
prob2.show_estado(initial2)

0: ██|░░
1: ██|██
2: ██

3: ░░|██
4: ██|██|░░
5: ░░|██


In [23]:
%%time
resuelve_contenedor(prob2, astar_search, prob2.heuristica)

In [21]:
initial3 = Estado([
     # Muelle 1
        [True, False],
        [True, False,],
        [False,],
     # Muelle 2
        [True, False,True],
        [True,False,True,],
        [True,False,],
])

In [35]:
prob3 = ProblemaContenedor(initial3, 3)
prob3.show_estado(initial3)

0: ░░|██
1: ░░|██
2: ██

3: ░░|██|░░
4: ░░|██|░░
5: ░░|██


In [36]:
%%time
resuelve_contenedor(prob3, astar_search, prob3.heuristica)

Solución: [Movimiento(origen=5, destino=2), Movimiento(origen=3, destino=2), Movimiento(origen=3, destino=1), Movimiento(origen=4, destino=2), Movimiento(origen=4, destino=1), Movimiento(origen=0, destino=1), Movimiento(origen=3, destino=2), Movimiento(origen=1, destino=3), Movimiento(origen=1, destino=3), Movimiento(origen=1, destino=3), Movimiento(origen=1, destino=3), Movimiento(origen=4, destino=2), Movimiento(origen=5, destino=2)]
Algoritmo: astar_search
Heurística: heuristica
Longitud de la solución: 13. Nodos analizados: 15


In [41]:
initial4 = Estado([
     # Muelle 1
        [True, False, True],
        [True, False, False, False, True],
        [False,],
     # Muelle 2
        [True, False,True],
        [True,False,True,],
        [True,False,],
])

In [42]:
prob4 = ProblemaContenedor(initial4, 3)
prob4.show_estado(initial4)

0: ░░|██|░░
1: ░░|██|██|██|░░
2: ██

3: ░░|██|░░
4: ░░|██|░░
5: ░░|██


In [43]:
%%time
resuelve_contenedor(prob4, astar_search, prob4.heuristica)

KeyboardInterrupt: KeyboardInterrupt: 

In [44]:
initial5 = Estado([
     # Muelle 1
        [],
        [],
        [],
     # Muelle 2
        [True, False,False, False,False, False,False, False,False, False,False, False,False, False,False, False,False, False,False],
        [True,False,True,],
        [True,False,],
])

In [45]:
prob5 = ProblemaContenedor(initial5, 3)
prob5.show_estado(initial5)

0: 
1: 
2: 

3: ░░|██|██|██|██|██|██|██|██|██|██|██|██|██|██|██|██|██|██
4: ░░|██|░░
5: ░░|██


In [46]:
%%time
resuelve_contenedor(prob5, astar_search, prob5.heuristica)

Solución: [Movimiento(origen=5, destino=2), Movimiento(origen=3, destino=2), Movimiento(origen=3, destino=2), Movimiento(origen=3, destino=2), Movimiento(origen=3, destino=2), Movimiento(origen=3, destino=2), Movimiento(origen=3, destino=2), Movimiento(origen=3, destino=2), Movimiento(origen=3, destino=2), Movimiento(origen=3, destino=2), Movimiento(origen=3, destino=1), Movimiento(origen=3, destino=1), Movimiento(origen=3, destino=1), Movimiento(origen=3, destino=1), Movimiento(origen=3, destino=1), Movimiento(origen=3, destino=1), Movimiento(origen=3, destino=1), Movimiento(origen=3, destino=1), Movimiento(origen=3, destino=1), Movimiento(origen=4, destino=1), Movimiento(origen=4, destino=0), Movimiento(origen=3, destino=0), Movimiento(origen=4, destino=0), Movimiento(origen=5, destino=0)]
Algoritmo: astar_search
Heurística: heuristica
Longitud de la solución: 24. Nodos analizados: 29


In [47]:
initial6 = Estado([
     # Muelle 1
        [],
        [],
        [],
     # Muelle 2
        [True, False],
        [True,False,True,],
        [True,False],
        [True,False],
        [True,False],
        [True,False],
        [True,False],
        [True,False],
        [True,False],
        [True,False],
        [True,False],
        [True,False],
        [True,False],
        [True,False],
        [True,False],
        [True,False],
        [True,False],
])

In [50]:
prob6 = ProblemaContenedor(initial6, 3)
prob6.show_estado(initial6)

 0: 
 1: 
 2: 

 3: ░░|██
 4: ░░|██|░░
 5: ░░|██
 6: ░░|██
 7: ░░|██
 8: ░░|██
 9: ░░|██
10: ░░|██
11: ░░|██
12: ░░|██
13: ░░|██
14: ░░|██
15: ░░|██
16: ░░|██
17: ░░|██
18: ░░|██
19: ░░|██


In [48]:
%%time
resuelve_contenedor(prob6, astar_search, prob6.heuristica)

NameError: NameError: name 'prob6' is not defined

# E. Comentar cómo afecta a la resolución del problema
## Variar el número de pilas

Cuando variamos el número de pilas **disminuímos o aumentamos el número de posibles movimientos**, porque los movimientos posibles se da mediante un producto cartesiano de pilas no vacías por pilas con hueco libre (con origen distintos del destino). Por lo que el factor de ramificación se ve modificado, y provoca que el árbol de estados aumenta o disminuye de manera exponencial.

La representación actual del problema es apto para este cambio, no hay que realizar cambios en la definición de la clase. Simplemente añadir más pilas en la definición del estado inicial. No podemos unificar los dos muelles objetivos directamente, porque quizá hay un coste por usar la cinta para transportar entre esos dos muelles.

## Incluir un muelle adicional

Si solo queremos incluir un nuevo muelle que no sea objetivo (es decir, en el estado final no puede haber contenedores objetivos allí), entonces nuestra representación actual del problema sigue valiendo. Pues sólo distinguimos entre el muelle 1 y el resto de muelles.

Sin embargo, si tuviesemos varios muelles objetivos, habría que modificar el código, para saber qué pilas se encuentra en muelle objetivo y ajustar la heurística de acuerdo con eso.